In [1]:
data_dir = 'D:/UpGrad_Notes/ljmu immersion/Research/interim report/Coding/nifti/FDG-PET-CT-Lesions'

In [2]:
import glob
import os

In [3]:
images_pt = sorted(glob.glob(os.path.join(data_dir, 'PETCT*', '*', "SUV*")))
images_ct = sorted(glob.glob(os.path.join(data_dir, 'PETCT*', '*', "CTres*")))
images_sg = sorted(glob.glob(os.path.join(data_dir, 'PETCT*', '*', "SEG*")))


In [4]:
data_dicts = [
            {"image_pt": image_name_pt, "image_ct": image_name_ct, "image_sg": image_name_sg}
            for image_name_pt, image_name_ct, image_name_sg in zip(images_pt, images_ct, images_sg)
        ]

In [5]:
data_dicts[0]

{'image_pt': 'D:/UpGrad_Notes/ljmu immersion/Research/interim report/Coding/nifti/FDG-PET-CT-Lesions\\PETCT_0011f3deaf\\03-23-2003-NA-PET-CT Ganzkoerper  primaer mit KM-10445\\SUV.nii.gz',
 'image_ct': 'D:/UpGrad_Notes/ljmu immersion/Research/interim report/Coding/nifti/FDG-PET-CT-Lesions\\PETCT_0011f3deaf\\03-23-2003-NA-PET-CT Ganzkoerper  primaer mit KM-10445\\CTres.nii.gz',
 'image_sg': 'D:/UpGrad_Notes/ljmu immersion/Research/interim report/Coding/nifti/FDG-PET-CT-Lesions\\PETCT_0011f3deaf\\03-23-2003-NA-PET-CT Ganzkoerper  primaer mit KM-10445\\SEG.nii.gz'}

In [6]:
data_dicts[:2]

[{'image_pt': 'D:/UpGrad_Notes/ljmu immersion/Research/interim report/Coding/nifti/FDG-PET-CT-Lesions\\PETCT_0011f3deaf\\03-23-2003-NA-PET-CT Ganzkoerper  primaer mit KM-10445\\SUV.nii.gz',
  'image_ct': 'D:/UpGrad_Notes/ljmu immersion/Research/interim report/Coding/nifti/FDG-PET-CT-Lesions\\PETCT_0011f3deaf\\03-23-2003-NA-PET-CT Ganzkoerper  primaer mit KM-10445\\CTres.nii.gz',
  'image_sg': 'D:/UpGrad_Notes/ljmu immersion/Research/interim report/Coding/nifti/FDG-PET-CT-Lesions\\PETCT_0011f3deaf\\03-23-2003-NA-PET-CT Ganzkoerper  primaer mit KM-10445\\SEG.nii.gz'},
 {'image_pt': 'D:/UpGrad_Notes/ljmu immersion/Research/interim report/Coding/nifti/FDG-PET-CT-Lesions\\PETCT_01140d52d8\\08-13-2005-NA-PET-CT Ganzkoerper  primaer mit KM-56839\\SUV.nii.gz',
  'image_ct': 'D:/UpGrad_Notes/ljmu immersion/Research/interim report/Coding/nifti/FDG-PET-CT-Lesions\\PETCT_01140d52d8\\08-13-2005-NA-PET-CT Ganzkoerper  primaer mit KM-56839\\CTres.nii.gz',
  'image_sg': 'D:/UpGrad_Notes/ljmu immersion

In [7]:
train_files, val_files = data_dicts[:2], data_dicts[-2:]

In [8]:
val_files

[{'image_pt': 'D:/UpGrad_Notes/ljmu immersion/Research/interim report/Coding/nifti/FDG-PET-CT-Lesions\\PETCT_ff1451316e\\03-31-2003-NA-PET-CT Ganzkoerper  primaer mit KM-22165\\SUV.nii.gz',
  'image_ct': 'D:/UpGrad_Notes/ljmu immersion/Research/interim report/Coding/nifti/FDG-PET-CT-Lesions\\PETCT_ff1451316e\\03-31-2003-NA-PET-CT Ganzkoerper  primaer mit KM-22165\\CTres.nii.gz',
  'image_sg': 'D:/UpGrad_Notes/ljmu immersion/Research/interim report/Coding/nifti/FDG-PET-CT-Lesions\\PETCT_ff1451316e\\03-31-2003-NA-PET-CT Ganzkoerper  primaer mit KM-22165\\SEG.nii.gz'},
 {'image_pt': 'D:/UpGrad_Notes/ljmu immersion/Research/interim report/Coding/nifti/FDG-PET-CT-Lesions\\PETCT_ff39795341\\09-22-2005-NA-PET-CT Ganzkoerper  primaer mit KM-98939\\SUV.nii.gz',
  'image_ct': 'D:/UpGrad_Notes/ljmu immersion/Research/interim report/Coding/nifti/FDG-PET-CT-Lesions\\PETCT_ff39795341\\09-22-2005-NA-PET-CT Ganzkoerper  primaer mit KM-98939\\CTres.nii.gz',
  'image_sg': 'D:/UpGrad_Notes/ljmu immersion

In [9]:
from monai.utils import set_determinism
from monai.data.utils import list_data_collate
from monai.data import PersistentDataset,DataLoader

In [10]:

# set deterministic training for reproducibility
set_determinism(seed=0)


In [11]:
import torch

from monai.transforms import (
    Compose,
    LoadImaged,
    ScaleIntensityRanged,
    ConcatItemsd,
    EnsureChannelFirstd,
    ToTensord,
    CropForegroundd,
    RandCropByPosNegLabeld    
)


In [12]:
        # define the data transforms
        train_transforms = Compose(
            [
                LoadImaged(keys=["image_pt", "image_ct", "image_sg"]),
                ScaleIntensityRanged(
                    keys=["image_ct"], a_min=-100, a_max=250,
                    b_min=0.0, b_max=1.0, clip=False,
                ),
                ScaleIntensityRanged(
                    keys=["image_pt"], a_min=0, a_max=15,
                    b_min=0.0, b_max=1.0, clip=False,
                ),
                ConcatItemsd(keys=["image_pt", "image_ct"], name="image_petct", dim=0),

                # AddChanneld(keys=["image_pt", "image_ct", "image_sg"]),
                # concatenate pet and ct channels
                EnsureChannelFirstd(keys=["image_petct", "image_sg"]),
                # Orientationd(keys=["image_petct", "image_sg"], axcodes="LAS"),
                # Spacingd(
                #     keys=["image_pt", "image_ct", "image_sg"],
                #     pixdim=(2, 2, 3),
                #     mode=("bilinear", "bilinear", "nearest"),
                # ),

                # user can also add other random transforms
                ToTensord(keys=["image_petct"]),

                # RandAffined(
                #     keys=["image_petct", "image_sg"],
                #     mode=("bilinear", "nearest"),
                #     prob=1.0,
                #     spatial_size=(64, 64, 64),
                #     rotate_range=(0, 0, np.pi / 15),
                #     scale_range=(0.1, 0.1, 0.1)),
                CropForegroundd(keys=["image_petct", "image_sg"], source_key="image_petct"),
                # randomly crop out patch samples from
                # big image based on pos / neg ratio
                # the image centers of negative samples
                # must be in valid image area
                RandCropByPosNegLabeld(
                    keys=["image_petct", "image_sg"],
                    label_key="image_sg",
                    spatial_size=(5, 5, 5),
                    pos=1,
                    neg=1,
                    num_samples=4,
                    image_key="image_petct",
                    image_threshold=0,
                ),

            ]
        )


C:\Users\jagdi\anaconda3\envs\pytorchenv\lib\site-packages\monai\utils\deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)


In [13]:
val_transforms = Compose(
            [
                LoadImaged(keys=["image_pt", "image_ct", "image_sg"]),
                # AddChanneld(keys=["image_pt", "image_ct", "image_sg"]),
                ScaleIntensityRanged(
                    keys=["image_ct"], a_min=-100, a_max=250,
                    b_min=0.0, b_max=1.0, clip=False,
                ),
                ScaleIntensityRanged(
                    keys=["image_pt"], a_min=0, a_max=15,
                    b_min=0.0, b_max=1.0, clip=False,
                ),
                ConcatItemsd(keys=["image_pt", "image_ct"], name="image_petct", dim=0),
                # concatenate pet and ct channels
                EnsureChannelFirstd(keys=["image_petct", "image_sg"]),

                # Spacingd(
                #  keys=["image_pt", "image_ct", "image_sg"],
                # pixdim=(2, 2, 3),
                # mode=("bilinear", "bilinear", "nearest"),
                # ),
                # Orientationd(keys=["image_pt", "image_ct", "image_sg"], axcodes="LAS"),


                CropForegroundd(keys=["image_petct", "image_sg"], source_key="image_petct"),

                ToTensord(keys=["image_petct"]),
            ]
        )

In [14]:
train_ds = PersistentDataset(
            data=train_files[:10], transform=train_transforms, cache_dir='cache_dir'
        )

In [15]:
train_loader = DataLoader(
            train_ds, batch_size=1, shuffle=True,
            num_workers=6, collate_fn=list_data_collate,
        )
